In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.express as px
import plotly.graph_objects as go
import umap
import warnings
warnings.filterwarnings("ignore")
import plotly.io as pio
pio.renderers.default='notebook'

/Users/tatiana.ilyasova/Library/Python/3.10/lib/python/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/tatiana.ilyasova/Library/Python/3.10/lib/python/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/tatiana.ilyasova/Library/Python/3.10/lib/python/site-

In [2]:
BASE_DIR = Path.cwd().parent

In [3]:
df = pd.read_parquet(BASE_DIR / 'processed_data' / 'hosts_reviews_en_labeled.parquet')

In [ ]:
df_hosts_reviews_en_two = df.query('neighbourhood == "Östermalm" or neighbourhood == "Enskede-Årsta-Vantör"')
df_hosts_reviews_en_two['comments'] = df_hosts_reviews_en_two['comments'].str.replace('enskede|årsta|vantörs|östermalm', '', regex=True)

In [5]:
def vectorizier_X(df, col):
    X = df[col]
    vectorizer = TfidfVectorizer(stop_words='english',
                             ngram_range=(1,2),
                            min_df=5,
                            max_df=0.5
                            )
    X_vect = vectorizer.fit_transform(X)
    return X_vect

In [10]:
def umap_viz(df, col, X_vect, colors, n=2, n_n=150, min_dist=0.9, metric='cosine'):
    umap_emb = umap.UMAP(n_components=n, n_neighbors=n_n, min_dist=min_dist, 
                        metric=metric, random_state=42).fit_transform(X_vect)
    trace = go.Scatter(
    x=umap_emb[:,0],
    y=umap_emb[:,1],
    mode='markers',
    text=df[col],
    marker=dict(
        size=5,
        color=colors,
        )
    )


    data = [trace]
    layout = go.Layout(title='UMAP Visualization')
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

In [11]:
X_vect = vectorizier_X(df_hosts_reviews_en_two, 'comments')
colors_dict = dict(zip(['Östermalm', 'Enskede-Årsta-Vantör'], ['tomato', 'steelblue']))
colors = [colors_dict[rt] for rt in df_hosts_reviews_en_two['neighbourhood']]
umap_viz(df_hosts_reviews_en_two, 'comments', X_vect, colors)